# Fine-tuning a model with the Trainer API or Keras

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]

🤗 Transformers provides a Trainer class to help you fine-tune any of the pretrained models it provides on your dataset. Once you've done all the data preprocessing work in the last section, you have just a few steps left to define the `Trainer`. The hardest part is likely to be preparing the environment to run `Trainer.train()`, as it will run very slowly on a CPU. If you don't have a GPU set up, you can get access to free GPUs or TPUs on Google Colab.

The code examples below assume you have already executed the examples in the previous section. Here is a short summary recapping what you need:

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

**Training**

The first step before we can define our `Trainer` is to define a `TrainingArguments` class that will contain all the hyperparameters the Trainer will use for training and evaluation. The only argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way. For all the rest, you can leave the defaults, which should work pretty well for a basic fine-tuning.

In [3]:
%%capture
!pip install transformers[torch] -U
!pip install accelerate -U

In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

**Note:** 💡 If you want to automatically upload your model to the Hub during training, pass along `push_to_hub=True` in the `TrainingArguments`. We will learn more about this in [Chapter 4](https://huggingface.co/course/chapter4/3).

---



The second step is to define our model. As in the [previous chapter](https://huggingface.co/course/chapter2), we will use the `AutoModelForSequenceClassification` class, with two labels:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

You will notice that unlike in [Chapter 2](https://huggingface.co/course/chapter2), you get a warning after instantiating this pretrained model. This is because BERT has not been pretrained on classifying pairs of sentences, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

Once we have our model, we can define a `Trainer` by passing it all the objects constructed up to now — the `model`, the `training_args`, the `training` and `validation` datasets, our `data_collator`, and our `tokenizer`:

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Note that when you pass the tokenizer as we did here, the default `data_collator` used by the `Trainer` will be a `DataCollatorWithPadding` as defined previously, so you can skip the line `data_collator=data_collator` in this call. It was still important to show you this part of the processing in section 2!

To fine-tune the model on our dataset, we just have to call the `train()` method of our Trainer:

In [ ]:
trainer.train()

This will start the fine-tuning (which should take a couple of minutes on a GPU) and report the training loss every 500 steps. It won't, however, tell you how well (or badly) your model is performing. This is because:

1. We didn't tell the `Trainer` to evaluate during training by setting `evaluation_strategy` to either "steps" (evaluate every `eval_steps`) or "epoch" (evaluate at the end of each epoch).
2. We didn't provide the `Trainer` with a `compute_metrics()` function to calculate a metric during said evaluation (otherwise the evaluation would just have printed the loss, which is not a very intuitive number).

**Evaluation**

Let's see how we can build a useful `compute_metrics()` function and use it the next time we train. The function must take an `EvalPrediction` object (which is a named tuple with a `predictions` field and a `label_ids` field) and will return a dictionary mapping strings to floats (the strings being the names of the metrics returned, and the floats their values). To get some predictions from our model, we can use the `Trainer.predict()` command:

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The output of the `predict()` method is another named tuple with three fields: `predictions`, `label_ids`, and `metrics`. The metrics field will just contain the loss on the dataset passed, as well as some time metrics (how long it took to predict, in total and on average). Once we complete our `compute_metrics()` function and pass it to the Trainer, that field will also contain the metrics returned by `compute_metrics()`.

As you can see, predictions is a two-dimensional array with shape 408 x 2 (408 being the number of elements in the dataset we used). Those are the logits for each element of the dataset we passed to `predict()` (as you saw in the [previous chapter](https://huggingface.co/course/chapter2), all Transformer models return logits). To transform them into predictions that we can compare to our labels, we need to take the index with the maximum value on the second axis:

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

We can now compare those preds to the labels. To build our `compute_metric()` function, we will rely on the metrics from the 🤗 [Evaluate library](https://github.com/huggingface/evaluate/). We can load the metrics associated with the MRPC dataset as easily as we loaded the dataset, this time with the `evaluate.load()` function. The object returned has a `compute()` method we can use to do the metric calculation:

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

The exact results you get may vary, as the random initialization of the model head might change the metrics it achieved. Here, we can see our model has an accuracy of 85.78% on the validation set and an F1 score of 89.97. Those are the two metrics used to evaluate results on the MRPC dataset for the GLUE benchmark. The table in the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf) reported an F1 score of 88.9 for the base model. That was the uncased model while we are currently using the cased model, which explains the better result.

Wrapping everything together, we get our `compute_metrics()` function:

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

And to see it used in action to report metrics at the end of each epoch, here is how we define a new `Trainer` with this `compute_metrics()` function:

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Note that we create a new `TrainingArguments` with its `evaluation_strategy` set to "epoch" and a new model — otherwise, we would just be continuing the training of the model we have already trained. To launch a new training run, we execute:

In [ ]:
trainer.train()

This time, it will report the validation loss and metrics at the end of each epoch on top of the training loss. Again, the exact accuracy/F1 score you reach might be a bit different from what we found, because of the random head initialization of the model, but it should be in the same ballpark.

The Trainer will work out of the box on multiple GPUs or TPUs and provides lots of options, like mixed-precision training (use fp16 = True in your training arguments). We will go over everything it supports in Chapter 10.

This concludes the introduction to fine-tuning using the `Trainer` API. An example of doing this for most common NLP tasks will be given in [Chapter 7](https://huggingface.co/course/chapter7), but for now let's look at how to do the same thing in pure PyTorch.

✏️ **Try it out!** Fine-tune a model on the GLUE SST-2 dataset, using the data processing you did in section 2.

**Loading the GLUE SST-2 dataset using the Hugging Face Transformers library:** We will go through the process of preprocessing the GLUE SST-2 dataset using the Hugging Face Transformers library. The SST-2 dataset is a single-sentence text classification task, making it slightly different from other GLUE tasks that involve pairs of sentences. We'll cover loading the dataset, tokenization, and dynamic padding.

**1. Loading the Dataset:**

We'll start by loading the SST-2 dataset from the 🤗 Datasets library. This dataset consists of single sentences along with their corresponding labels.

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")

**2. Tokenization and Preprocessing:**

Now that we have the raw dataset, let's preprocess the data by tokenizing the sentences using a pretrained tokenizer. We'll define a tokenization function and then apply it to the entire dataset.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

# Tokenize the entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

**3. Dynamic Padding:**

Dynamic padding allows us to pad the batch to the length of the longest sequence within that batch, instead of padding to the maximum sequence length in the entire dataset. This improves efficiency by reducing unnecessary padding.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Example: Select a few samples from the training set
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence"]}

# Apply dynamic padding using data_collator
batch = data_collator(samples)

{k: v.shape for k, v in batch.items()}

The batch dictionary now contains keys for `input_ids`, `attention_mask`, `token_type_ids`, and `labels`, each corresponding to a PyTorch tensor. The `attention_mask` indicates which tokens are padding tokens, and `token_type_ids` distinguish between `sentences` in paired tasks.

**4. Fine-Tuning:**

With the preprocessed data, you can now proceed to fine-tune your model on the SST-2 dataset. Utilize the `batch` dictionary as input to your model for training and validation.

This completes the preprocessing of the GLUE SST-2 dataset using Hugging Face Transformers. The combination of tokenization and dynamic padding allows you to efficiently prepare your data for fine-tuning and training classification models on various NLP tasks.

**Full Sequence: Fine-Tuning a Model on GLUE SST-2 Dataset using Hugging Face Transformers**

In this part, we will walk through the process of fine-tuning a model on the GLUE SST-2 (Single-Sentence Text Classification) dataset using the Hugging Face Transformers library. We will cover defining a Trainer, and evaluating the model's performance.

**1. Load dataset:**

First, we need to preprocess the dataset using the steps provided in the previous part of the tutorial. This involves loading the GLUE SST-2 dataset, tokenizing the sentences, and applying dynamic padding.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "sst2")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**2. Defining TrainingArguments:**

Now, let's define the training arguments for the Trainer. These arguments will include the directory to save the trained model and other hyperparameters. We will use the TrainingArguments class for this.

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("sst2-finetuned-model")

**3. Defining the Model:**

Next, we'll define the model to be fine-tuned. We'll use the AutoModelForSequenceClassification class and specify the number of labels (in this case, 2 for binary classification).

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

**4. Defining the Trainer:**

Now, we can define the Trainer using the model, training arguments, datasets, data collator, and tokenizer. Additionally, we'll specify the `compute_metrics` function to evaluate the model's performance during training.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Define your custom metrics function
    tokenizer=tokenizer,
)

**5. Fine-Tuning the Model:**

With the Trainer defined, we can now initiate the fine-tuning process using the `train()` method.

In [ ]:
trainer.train()

The Trainer will take care of training the model, reporting losses, and evaluating metrics during the training process. Make sure to have access to a GPU or TPU for faster training.

**6. Evaluating the Model:**

After training is complete, you can evaluate the model's performance on the evaluation dataset. The Trainer's `evaluate()` method can be used to do this.

In [ ]:
evaluation_results = trainer.evaluate(tokenized_datasets["validation"])
print(evaluation_results)

This will give you a dictionary of evaluation results, which can include metrics like accuracy, F1-score, etc.

**7. Saving the Fine-Tuned Model:**

To save the fine-tuned model, you can use the `save_model()` method provided by the Trainer.

In [ ]:
trainer.save_model("fine_tuned_sst2_model")

Now you have successfully fine-tuned a model on the GLUE SST-2 dataset using the Hugging Face Transformers library. This process can be adapted to other NLP tasks and datasets as well, allowing you to leverage the power of pre-trained transformer models for your specific applications.